# Define the taxonomy

In [1]:
# For simplification, the following are used to represent the nodes (on p1752 of Landy & Bartels):
# ca => care
# fa => fairness
# au => authority
# lo => loyalty
# po => politeness
# pr => propriety
# sa => sanctity
# li => liberty
# all => all moral virtues
level1 = ['ca', 'fa', 'au', 'lo', 'po', 'pr', 'sa', 'li', 'all']

# generate level 2 of the taxonomy, with each element being a list containing all the individual nodes
# note that we can't have 2 nodes with overlap (i.e. Propriety and Care), and we can't have all moral virtues
level2 = []
all_propriety = ['ca', 'fa', 'au', 'lo', 'po']
for i in range(len(all_propriety)):
    n1 = all_propriety[i]
    # append the two nodes directly under all moral virtues
    level2.append([n1,'sa'])
    level2.append([n1,'li'])
    # add all possible combinations of this node plus others in propriety
    for j in range(i+1,len(all_propriety)):
        level2.append([n1,all_propriety[j]])
    
# generate level 3 of the taxonomy
# all of them are just one thing under propriety + sanctity and liberty
level3 = [['pr','sa','li']]
for n1 in all_propriety:
    level3.append([n1,'sa','li'])

# We could keep going up, but will stop here for now. 
print(level1)
print(level2)
print(level3)

['ca', 'fa', 'au', 'lo', 'po', 'pr', 'sa', 'li', 'all']
[['ca', 'sa'], ['ca', 'li'], ['ca', 'fa'], ['ca', 'au'], ['ca', 'lo'], ['ca', 'po'], ['fa', 'sa'], ['fa', 'li'], ['fa', 'au'], ['fa', 'lo'], ['fa', 'po'], ['au', 'sa'], ['au', 'li'], ['au', 'lo'], ['au', 'po'], ['lo', 'sa'], ['lo', 'li'], ['lo', 'po'], ['po', 'sa'], ['po', 'li']]
[['pr', 'sa', 'li'], ['ca', 'sa', 'li'], ['fa', 'sa', 'li'], ['au', 'sa', 'li'], ['lo', 'sa', 'li'], ['po', 'sa', 'li']]


# Define functions

### Helper functions

In [2]:
# get the layer number for the given hypothesis
def get_layer(hypothesis):
    # define the hypothesis as a set, regardless of length
    if len(hypothesis) == 1:
        if hypothesis[0] in level1:
            return 1
        else:
            return -1
        
    if len(hypothesis) == 2:
        for hyp in level2:
            if hypothesis[0] in hyp and hypothesis[1] in hyp:
                return 2
        return -1
    
    if len(hypothesis) == 3:
        for hyp in level3:
            if hypothesis[0] in hyp and hypothesis[1] in hyp and hypothesis[2] in hyp:
                return 3
        return -1

# get the size of the hypothesis, in terms of number of lowest-level categories
def get_hypothesis_size(h):
    # 7 total leaf nodes
    if 'all' in h:
        return 7
    
    # compute size if not all
    size = len(h)
    # if proprietary is in the hypothesis, replace with the 5 sub-categories (leaf nodes)
    if 'pr' in h:
        size = size + 4
    return size

# check whether all elements of the hypothesis are contained in X
def contains_all(h,X):
    if 'all' in h:
        return True
    
    for item in X:
        # check if the item is contained in h
        if item in h:
            pass
        elif item in all_propriety and 'pr' in h:
            pass
        else:
            return False
    
    # if no missing elements, return true
    return True

def is_in(y,h):
    return 'all' in h or y in h or (y in all_propriety and 'pr' in h)

### Functions to compute priors & likelihoods

In [3]:
# compute the prior, given lambda and h
def compute_prior(lambda_param,h):
    k = get_layer(h)
    return (lambda_param / (1 - lambda_param))**k

# compute the likelihood, given X and h
def compute_likelihood(X,h):
    # X is a list of individual examples (i.e. leaf nodes); h may have some higher level nodes as well
    if contains_all(h,X):
        n = len(X)
        s_h = get_hypothesis_size(h)
        return (1/s_h)**n
    return 0

### Compute P(y in C)

In [4]:
all_hypotheses = level1 + level2 + level3

def compute_all(X,lambda_param):
    total = 0
    for h in all_hypotheses:
        total = total + compute_likelihood(X,h) * compute_prior(lambda_param,h)
    return total

# Compute P(h|X)
def compute_h_given_X(h,X,lambda_param):
    return compute_likelihood(X,h) * compute_prior(lambda_param,h) / compute_all(X,lambda_param)

# Compute P(y in C)
def compute_y_in_C(y,X,lambda_param):
    p = 0
    for h in all_hypotheses:
        if is_in(y,h):
            p = p + compute_h_given_X(h,X,lambda_param)
    return p

# Intermediate Tests

In [5]:
# Example: Joe has committed care & fairness violations. How likely is he to commit a sanctity violation? 
# X = [care, fairness]
# y = sanctity
print(compute_y_in_C('sa',['ca','fa'],0.5))

0.1281138790035587


In [6]:
# Example: Joe has committed care & fairness violations. How likely is he to commit a loyalty violation? 
# X = [care, fairness]
# y = loyalty
print(compute_y_in_C('lo',['ca','ca'],0.5))
# We would expect this probability to be higher than the previous one, because both are violations of propriety. 

0.16509988249118682


In [7]:
# Note that these numbers are not technically probabilities since we did not define P(X). Thus, it is their values
# relative to each other that matter, not the actual values themselves. 

### Compute an actual probability

In [8]:
# compute probabilities for all possible y-values given a specific X and lambda parameter
def compute_all_probabilities(X,lambda_param):
    all_y = ['ca', 'fa', 'au', 'lo', 'po', 'sa', 'li']
    probs = []
    for y in all_y:
        probs.append([y,compute_y_in_C(y,X,lambda_param)])
    return probs

# Tests for Actual Probability

In [9]:
# Testing single vs multiple examples of a single subcategory (care)
print('Single example:\n',compute_all_probabilities(['ca'],0.1))
print('Multiple examples:\n',compute_all_probabilities(['ca','ca','ca'],0.1))

Single example:
 [['ca', 1.0000000000000002], ['fa', 0.38125479422730446], ['au', 0.38125479422730446], ['lo', 0.38125479422730446], ['po', 0.38125479422730446], ['sa', 0.17649446099505867], ['li', 0.17649446099505867]]
Multiple examples:
 [['ca', 1.0], ['fa', 0.0577705865363241], ['au', 0.0577705865363241], ['lo', 0.0577705865363241], ['po', 0.0577705865363241], ['sa', 0.023154735516379785], ['li', 0.023154735516379785]]


In [10]:
# Testing with 2 examples, in same vs different categories (all under propriety)
print('Same category:\n',compute_all_probabilities(['ca'],0.1))
print('Multiple categories:\n',compute_all_probabilities(['ca','fa'],0.1))

Same category:
 [['ca', 1.0000000000000002], ['fa', 0.38125479422730446], ['au', 0.38125479422730446], ['lo', 0.38125479422730446], ['po', 0.38125479422730446], ['sa', 0.17649446099505867], ['li', 0.17649446099505867]]
Multiple categories:
 [['ca', 1.0], ['fa', 1.0], ['au', 0.992933824707579], ['lo', 0.992933824707579], ['po', 0.992933824707579], ['sa', 0.42057362602147086], ['li', 0.42057362602147086]]


In [11]:
# Testing with 2 examples, in same vs different categories (all propriety or propriety/sanctity/liberty)
print('Same category:\n',compute_all_probabilities(['ca','fa','au'],0.1))
print('Multiple categories:\n',compute_all_probabilities(['ca','sa','li'],0.1))

Same category:
 [['ca', 1.0], ['fa', 1.0], ['au', 1.0], ['lo', 1.0], ['po', 1.0], ['sa', 0.3864404284294601], ['li', 0.3864404284294601]]
Multiple categories:
 [['ca', 1.0], ['fa', 0.9980677906904691], ['au', 0.9980677906904691], ['lo', 0.9980677906904691], ['po', 0.9980677906904691], ['sa', 1.0], ['li', 1.0]]


In [12]:
# Testing with 2 examples, in same vs different categories (all propriety or propriety/sanctity/liberty)
# Lambda has been changed to 0.5 (larger)
print('Same category:\n',compute_all_probabilities(['ca','fa','au'],0.5))
print('Multiple categories:\n',compute_all_probabilities(['ca','sa','li'],0.5))

Same category:
 [['ca', 0.9999999999999999], ['fa', 0.9999999999999999], ['au', 0.9999999999999999], ['lo', 0.9999999999999999], ['po', 0.9999999999999999], ['sa', 0.5574193548387096], ['li', 0.5574193548387096]]
Multiple categories:
 [['ca', 1.0], ['fa', 0.13602015113350127], ['au', 0.13602015113350127], ['lo', 0.13602015113350127], ['po', 0.13602015113350127], ['sa', 1.0], ['li', 1.0]]


In [13]:
# Testing with 2 examples, in same vs different categories (all propriety or propriety/sanctity/liberty)
# Lambda changed to 0.45 (slightly smaller than previous test)
print('Same category:\n',compute_all_probabilities(['ca','fa','au'],0.45))
print('Multiple categories:\n',compute_all_probabilities(['ca','sa','li'],0.45))

Same category:
 [['ca', 1.0], ['fa', 1.0], ['au', 1.0], ['lo', 1.0], ['po', 1.0], ['sa', 0.47697072174697797], ['li', 0.47697072174697797]]
Multiple categories:
 [['ca', 1.0], ['fa', 0.20279098239136886], ['au', 0.20279098239136886], ['lo', 0.20279098239136886], ['po', 0.20279098239136886], ['sa', 1.0], ['li', 1.0]]


## Model Predictions for Experiment

In [17]:
# 1-example predictions
# for 1 propriety: should I change this to the subcategory, ex "lo"? would be the same for all subcategories. 
print('Premise: 1 propriety\n',compute_all_probabilities(['lo'],0.1)) 
print('\nPremise: 1 liberty\n',compute_all_probabilities(['li'],0.1))
print('\nPremise: 1 sanctity\n',compute_all_probabilities(['sa'],0.1))

Premise: 1 propriety
 [['ca', 0.38125479422730446], ['fa', 0.38125479422730446], ['au', 0.38125479422730446], ['lo', 1.0000000000000002], ['po', 0.38125479422730446], ['sa', 0.17649446099505867], ['li', 0.17649446099505867]]

Premise: 1 liberty
 [['ca', 0.2221218176103453], ['fa', 0.2221218176103453], ['au', 0.2221218176103453], ['lo', 0.2221218176103453], ['po', 0.2221218176103453], ['sa', 0.2213753311481299], ['li', 1.0]]

Premise: 1 sanctity
 [['ca', 0.2221218176103453], ['fa', 0.2221218176103453], ['au', 0.2221218176103453], ['lo', 0.2221218176103453], ['po', 0.2221218176103453], ['sa', 1.0], ['li', 0.2213753311481299]]


In [15]:
# 3-example predictions
print('Premise: 3 propriety (same)\n',compute_all_probabilities(['lo','lo','lo'],0.1)) # same for all subcategories (just sub.)
print('\nPremise: 3 propriety (different)\n',compute_all_probabilities(['ca','fa','au'],0.1)) # same for any 3 subcategories
print('\nPremise: 3 liberty\n',compute_all_probabilities(['li','li','li'],0.1))
print('\nPremise: 3 sanctity\n',compute_all_probabilities(['sa','sa','sa'],0.1))

Premise: 3 propriety (same)
 [['ca', 0.0577705865363241], ['fa', 0.0577705865363241], ['au', 0.0577705865363241], ['lo', 1.0], ['po', 0.0577705865363241], ['sa', 0.023154735516379785], ['li', 0.023154735516379785]]

Premise: 3 propriety (different)
 [['ca', 1.0], ['fa', 1.0], ['au', 1.0], ['lo', 1.0], ['po', 1.0], ['sa', 0.3864404284294601], ['li', 0.3864404284294601]]

Premise: 3 liberty
 [['ca', 0.024013773166334868], ['fa', 0.024013773166334868], ['au', 0.024013773166334868], ['lo', 0.024013773166334868], ['po', 0.024013773166334868], ['sa', 0.022857826279357504], ['li', 0.9999999999999997]]

Premise: 3 sanctity
 [['ca', 0.024013773166334868], ['fa', 0.024013773166334868], ['au', 0.024013773166334868], ['lo', 0.024013773166334868], ['po', 0.024013773166334868], ['sa', 0.9999999999999997], ['li', 0.022857826279357504]]
